In [1]:
import pandas as pd
from datetime import datetime
import datetime
import numpy as np
import re
from os import listdir
from tqdm import tqdm
from fuzzywuzzy import process

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Functions

In [3]:
def assign_date_partnership(organizacion):
    dict_poynters = {'africa_check':datetime.date(2019, 8, 15), 
                    'la_silla_vacia':datetime.date(2020, 1, 31),
                     'chequeado':datetime.date(2018, 5, 1),
                     'animalpolitico':datetime.date(2018, 3, 1),
                     'colombiacheck': datetime.date(2020, 9, 28),
                     'ecuador_chequea':datetime.date(2020, 4, 1),
                     'larepublica':datetime.date(2020, 6, 1),
                    }
    
    if organizacion in dict_poynters.keys():
        return(dict_poynters[organizacion])
    else:
        return(np.nan)

def poynter_any_time(organizacion):
    dict_poynters = {'africa_check':datetime.date(2019, 8, 15), 
                    'la_silla_vacia':datetime.date(2020, 1, 31),
                     'chequeado':datetime.date(2018, 5, 1),
                     'animalpolitico':datetime.date(2018, 3, 1),
                     'colombiacheck': datetime.date(2020, 9, 28),
                     'ecuador_chequea':datetime.date(2020, 4, 1),
                     'larepublica':datetime.date(2020, 6, 1),
                    }
    
    if organizacion in dict_poynters.keys():
        return(1)
    else:
        return(0)
    
    
def title_factcheck_function(x):
    if pd.isna(x):
        return(None)
    else:
        return(re.sub('[/]$', '', x).split('/')[-1:][0])

def clean_links(df):
    final_links = []
    for i in list(df['Final Link']):
        if pd.isna(i):
            final_links.append(np.nan)
        elif any(re.findall('https://www.facebook.com/login/|https://api.whatsapp.com/',i)):
            final_links.append(np.nan)
        else:
            final_links.append(i)
    df['Final Link'] = final_links
    df = df[~df['Final Link'].isna()]
    df = df[df['Final Link'] != 'https://lasillavacia.com/']
    return(df)

def match_factcheck_w_posts(df, df_posts):
    date_factchecks = []
    match_link = []
    score_match_link = []
    match_title = []
    score_match_title = []
    post_url = []
    
    df = df.reset_index().drop('index', axis = 1)
    df['title_factcheck'] = df.apply(lambda x: title_factcheck_function(x['link_chequeo']), axis = 1)
    df_posts['title_link'] = df_posts.apply(lambda x: title_factcheck_function(x['Final Link']), axis = 1)
    for i in tqdm(range(0, len(df))):
        if pd.isna(df.loc[i]['date_factcheck_facebook']):
            link_factcheck = df.loc[i]['link_chequeo']
            
            if pd.isna(link_factcheck):
                    score_match_link.append(None)
                    match_link.append(None)
                    date_factchecks.append(None)
                    match_title.append(None)
                    score_match_title.append(None)
                    post_url.append(None)
            else:
                match, score = process.extractOne(link_factcheck, list(df_posts['Final Link']))
                if score >= 90:
                    score_match_link.append(score)
                    match_link.append(match)
                    date_factchecks.append(list(df_posts[df_posts['Final Link'] == match].Created)[0])
                    match_title.append(None)
                    score_match_title.append(None)
                    post_url.append(list(df_posts[df_posts['Final Link'] == match].URL)[0])
                else:
                    title_factcheck = df.loc[i]['title_factcheck']
                    match, score = process.extractOne(title_factcheck, list(df_posts['title_link']))
                    if score > 80:
                        score_match_link.append(None)
                        match_link.append(None)
                        date_factchecks.append(list(df_posts[df_posts['title_link'] == match].Created)[0])
                        match_title.append(match)
                        score_match_title.append(score)
                        post_url.append(list(df_posts[df_posts['title_link'] == match].URL)[0])
                    else:
                        score_match_link.append(None)
                        match_link.append(None)
                        date_factchecks.append(None)
                        match_title.append(None)
                        score_match_title.append(None)
                        post_url.append(None)
        else:
            score_match_link.append(None)
            match_link.append(None)
            date_factchecks.append(df.loc[i]['date_factcheck_facebook'])
            match_title.append(None)
            score_match_title.append(None)
            post_url.append(None)
            
    df['date_factcheck_facebook'] = date_factchecks
    df['match_link_factcheck'] = match_link
    df['score_match_link_factcheck'] = score_match_link
    df['match_title_factcheck'] = match_title
    df['score_match_title_factcheck'] = score_match_title
    df['post_url'] = post_url
    return(df)    

## Load Data

In [4]:
dates_chequeos = pd.read_json('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/06-virality/newspapers/2-id_data/dates_verificaciones.json')
id_data = pd.read_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/06-virality/newspapers/2-id_data/virality_id_newspapers.xlsx')

data_factchecks = id_data[['label_desinformacion', 'id_chequeo', 'organizacion', 
        'link_chequeo', 'fecha_chequeo', 'id_desinformacion']]
data_factchecks = data_factchecks.drop_duplicates()
data_factchecks['date_factcheck_facebook'] = None

posts_facebook = []
path = '/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/06-virality/factcheckers/posts_facebook/'
for i in listdir(path):
    try:
        df = pd.read_csv(path + i)
        df['organizacion'] = re.sub('[.]csv','', i)
        posts_facebook.append(df)
    except:
        print(i)
        
posts_facebook = pd.concat(posts_facebook)

.DS_Store


In [ ]:
matches_data = []

for i in tqdm(list(data_factchecks.organizacion.unique())[2:][7:]):
    if i == 'Chequeado-LATAM':
        df = data_factchecks[data_factchecks['organizacion'] == 'Chequeado-LATAM']
        df_posts = posts_facebook[posts_facebook['organizacion'] == 'chequeado']
        df_posts = clean_links(df_posts)
        input_matches = df_posts[['Final Link', 'URL', 'Created']].dropna()
        out = match_factcheck_w_posts(df, input_matches)
        matches_data.append(out)
    else:        
        df = data_factchecks[data_factchecks['organizacion'] == i]
        df_posts = posts_facebook[posts_facebook['organizacion'] == i]     
        df_posts = clean_links(df_posts)
        input_matches = df_posts[['Final Link', 'URL', 'Created']].dropna()
        out = match_factcheck_w_posts(df, input_matches)
        matches_data.append(out)

df_out = pd.concat(matches_data)

df_out.to_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/06-virality/newspapers/2-id_data/data_to_check.xlsx')

df_out = df_out[df_out['organizacion'] != 'la_silla_vacia']

        df = data_factchecks[data_factchecks['organizacion'] == 'la_silla_vacia']
        df_posts = posts_facebook[posts_facebook['organizacion'] == 'la_silla_vacia']
        df_posts = clean_links(df_posts)
        input_matches = df_posts[['Final Link', 'URL', 'Created']].dropna()
        out = match_factcheck_w_posts(df, input_matches)
        

df_clean = pd.concat([df_out, out])

data_ids = data_factchecks[['id_chequeo', 'id_desinformacion']].drop_duplicates().set_index('id_chequeo')['id_desinformacion'].to_dict()

df_clean = df_clean[df_clean['label_desinformacion'] == 'fake']
df_clean['id_desinformacion'] = df_clean['id_chequeo'].apply(lambda x: data_ids[x])
df_out_factchecks = df_clean[['organizacion','id_chequeo', 'fecha_chequeo', 'post_url','label_desinformacion', 'id_desinformacion',
       'date_factcheck_facebook', 'link_chequeo','match_link_factcheck', 'score_match_link_factcheck', 
       'title_factcheck','match_title_factcheck', 'score_match_title_factcheck','label_desinformacion']].rename(columns = {'link_chequeo':'link_factcheck', 
                                                                                                                          'organizacion':'factchecker', 
                                                                                                                          'fecha_chequeo':'date_factcheck',
                                                                                                                          'id_chequeo':'id_factcheck', 
                                                                                                                          'post_url':'post_factcheck'})



df_out_factchecks['poynter_facebook'] = df_out_factchecks.apply(lambda x: poynter_any_time(x['factchecker']), axis = 1)
df_out_factchecks['date_partnership_facebook'] = df_out_factchecks.apply(lambda x: assign_date_partnership(x['factchecker']), axis = 1)

df_out_factchecks['no_factcheck'] = np.where(pd.isna(df_out_factchecks['id_factcheck']), 1, 0)

df_out_factchecks['no_socialmedia_factcheck'] = None
df_out_factchecks['date_factcheck_twitter'] = None

factchekers = df_out_factchecks[['factchecker',
                                 'id_desinformacion', 
                                 'post_factcheck',
              'id_factcheck', 
              'date_factcheck',
              'date_factcheck_facebook', 
              'date_factcheck_twitter', 
              'label_desinformacion', 
              'no_factcheck',
              'no_socialmedia_factcheck', 
                  'link_factcheck',
            'match_link_factcheck', 'score_match_link_factcheck', 
       'title_factcheck','match_title_factcheck', 'score_match_title_factcheck']]

factchekers.to_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/06-virality/newspapers/2-id_data/factchekers_check2.xlsx')